# Forex Cordapp

In this section we show how to build a simple Cordapp where we can buy and sell currencies. Instead of building the whole application from scratch, we use the one available at [https://github.com/dineshrivankar/cordapp-trading/](https://github.com/dineshrivankar/cordapp-trading/).

The process of configuring the nodes is similar as in the Gradle example in the previous notebook. In this notebook we focus on the contracts and workflow implementation. This is the major part of each Cordapp.

## Configuration

Similar to the previous example, we need to set the ``build.gradle`` configuration file. The first part of the configuration file looks very similar to the previous one. Compared to the previous example, in the current we have the tests added.

In [3]:
%%writefile /home/codete/workshop/cordapp-trading/kotlin-source/build.gradle
repositories {
    mavenLocal()
    jcenter()
    mavenCentral()
    maven { url 'https://dl.bintray.com/kotlin/exposed' }
    maven { url 'https://jitpack.io' }
    maven { url 'https://ci-artifactory.corda.r3cev.com/artifactory/corda-releases' }
}

apply plugin: 'kotlin'
apply plugin: 'net.corda.plugins.cordapp'
apply plugin: 'net.corda.plugins.cordformation'
apply plugin: 'net.corda.plugins.quasar-utils'

jar.baseName = "cordapp-trading"

sourceSets {
    main {
        resources {
            srcDir "../config/dev"
        }
    }
    test {
        resources {
            srcDir "../config/test"
        }
    }
    integrationTest {
        kotlin {
            compileClasspath += main.output + test.output
            runtimeClasspath += main.output + test.output
            srcDir file('src/integration-test/kotlin')
        }
    }
}

configurations {
    integrationTestCompile.extendsFrom testCompile
    integrationTestRuntime.extendsFrom testRuntime
}

dependencies {
    compile "org.jetbrains.kotlin:kotlin-stdlib-jre8:$kotlin_version"
    testCompile "org.jetbrains.kotlin:kotlin-test:$kotlin_version"
    testCompile "junit:junit:$junit_version"

    // Corda integration dependencies
    cordaCompile "$corda_release_group:corda-core:$corda_release_version"
    cordaCompile "$corda_release_group:corda-finance:$corda_release_version"
    cordaCompile "$corda_release_group:corda-jackson:$corda_release_version"
    cordaCompile "$corda_release_group:corda-rpc:$corda_release_version"
    cordaCompile "$corda_release_group:corda-webserver-impl:$corda_release_version"
    cordaRuntime "$corda_release_group:corda:$corda_release_version"
    cordaRuntime "$corda_release_group:corda-webserver:$corda_release_version"

    testCompile "$corda_release_group:corda-node-driver:$corda_release_version"

    // CorDapp dependencies
    // Specify your cordapp's dependencies below, including dependent CorDapps
    cordapp "$corda_release_group:corda-finance:$corda_release_version"
}

task integrationTest(type: Test, dependsOn: []) {
    testClassesDirs = sourceSets.integrationTest.output.classesDirs
    classpath = sourceSets.integrationTest.runtimeClasspath
}

tasks.withType(org.jetbrains.kotlin.gradle.tasks.KotlinCompile).all {
    kotlinOptions {
        languageVersion = "1.1"
        apiVersion = "1.1"
        jvmTarget = "1.8"
        javaParameters = true   // Useful for reflection.
    }
}


Overwriting /home/codete/workshop/cordapp-trading/kotlin-source/build.gradle


Similar the ``deployNodes`` task from the previous notebook, the configuration has three nodes and the notary. The additional parameter is the ``webPort`` that allows to access the web application that is used to connect to one of our nodes. We show the web application implementation later in this notebook. 

In [4]:
%%writefile -a /home/codete/workshop/cordapp-trading/kotlin-source/build.gradle

task deployNodes(type: net.corda.plugins.Cordform, dependsOn: ['jar']) {
    directory "./build/nodes"
    node {
        name "O=Notary,L=London,C=GB"
        notary = [validating : false]
        p2pPort 10006
        cordapps = ["$corda_release_group:corda-finance:$corda_release_version"]
    }
    node {
        name "O=PartyA,L=Krakow,C=PL"
        p2pPort 10007
        rpcSettings {
            address("localhost:10008")
            adminAddress("localhost:10048")
        }
        webPort 10021
        cordapps = ["$corda_release_group:corda-finance:$corda_release_version"]
        rpcUsers = [[user: "user1", "password": "test", "permissions": ["ALL"]]]
    }
    node {
        name "O=PartyB,L=New York,C=US"
        p2pPort 10010
        rpcSettings {
            address("localhost:10011")
            adminAddress("localhost:10051")
        }
        webPort 10022
        cordapps = ["$corda_release_group:corda-finance:$corda_release_version"]
        rpcUsers = [[user: "user1", "password": "test", "permissions": ["ALL"]]]
    }
    node {
        name "O=PartyC,L=Paris,C=FR"
        p2pPort 10013
        rpcSettings {
            address("localhost:10014")
            adminAddress("localhost:10054")
        }
        webPort 10023
        cordapps = ["$corda_release_group:corda-finance:$corda_release_version"]
        rpcUsers = [[user: "user1", "password": "test", "permissions": ["ALL"]]]
    }
}

task runExampleClientRPCKotlin(type: JavaExec) {
    classpath = sourceSets.main.runtimeClasspath
    main = 'com.trading.client.ExampleClientRPCKt'
    args 'localhost:10008'
}

Appending to /home/codete/workshop/cordapp-trading/kotlin-source/build.gradle


Similar as in previous example, to generate the configuration and compile the Kotlin code we need to execute Gradle and the ``deployNodes`` task.

In [ ]:
!cd /home/codete/workshop/cordapp-trading/ && ./gradlew deployNodes

## Structure

The application structure consist of several directories, where the code in each is responsible for a separate functionality. To see what kind of directories we have we can use the command below. Please keep in mind that we don't need to have all directories and code to develop a working Corda application.

In [1]:
!ls /home/codete/workshop/cordapp-trading/kotlin-source/src/main/kotlin/com/trading/

api  client  contract  flow  plugin  schema  state


### Schema

Let's start with the Schema as it's the most crucial one. We define in the schema the structure of the data that each contract use. In the example we have the an table called ``trade_states`` that consist of fields like: ``initiatingPartyName``, ``counterParty``, ``sellValue``, ``sellCurrency``, ``buyValue``, ``buyCurrency``, ``tradeStatus``, and ``linearId``. The field defined in this schema are used by our application to make the buy/sell transaction possible. The schema definition can be fond in ``cordapp-trading\kotlin-source\src\main\kotlin\com\trading\schema\TradeSchema.kt``. The Entity definition looks as below.

```java
    @Entity
    @Table(name = "trade_states")
    class PersistentTrade(
            @Column(name = "initiatingParty")
            var initiatingPartyName: String,

            @Column(name = "counterParty")
            var counterParty: String,

            @Column(name = "sellValue")
            var sellValue: Int,

            @Column(name = "sellCurrency")
            var sellCurrency: String,

            @Column(name = "buyValue")
            var buyValue: Int,

            @Column(name = "buyCurrency")
            var buyCurrency: String,

            @Column(name = "tradeStatus")
            var tradeStatus: String,

            @Column(name = "linear_id")
            var linearId: UUID

    ) : PersistentState() {
        // Default constructor required by hibernate.
        constructor(): this("", "", 0,"",0,"","", UUID.randomUUID())
    }
```

### State

The state use the Schema and participants to record the agreement between two parties. The definition of the state can be found in ``cordapp-trading\kotlin-source\src\main\kotlin\com\trading\state\TradeState.kt``. We can see that our state implements two interfaces: ``LinearState`` and ``QueryableState``, where:

- ``LinearState`` - a state that evolves by superseding itself, all of which share the common "linearId",
- ``QueryableState`` - a contract state that may be mapped to database schemas configured for this node to support querying for, or filtering of, states.

The participants are set as a list of initiatingPart and counterParty. Both needs to agree to make the agreement possible.

```java
participants: List<AbstractParty> get() = listOf(initiatingParty, counterParty)
```

The implemented method ``generateMappedObject`` creates a ``PersistentState`` object with the schema information of the agreement.

```java
fun generateMappedObject(schema: MappedSchema): PersistentState {
        return when (schema) {
            is TradeSchemaV1 -> TradeSchemaV1.PersistentTrade(
                    this.initiatingParty.name.toString(),
                    this.counterParty.name.toString(),
                    this.sellValue,
                    this.sellCurrency.toString(),
                    this.buyValue,
                    this.buyCurrency.toString(),
                    this.tradeStatus,
                    this.linearId.id
            )
            else -> throw IllegalArgumentException("Unrecognised schema $schema")
        }
}
```

### Contract

The contract is responsible for the logic part. The main part of the contract is the ``verify`` method that checks if the states. The definition of the contract is placed in ``cordapp-trading\kotlin-source\src\main\kotlin\com\trading\contract\TradeContract.kt``. The ``verify`` method checks the two commands of creation and countertrade creation. The first has a list of requirement that needs to be fullfield to make it a valid operation:
```java
                requireThat {
                    // Generic constraints around the Trade transaction.
                    "No inputs should be consumed when issuing an Trade." using (tx.inputs.isEmpty())
                    "Only one output state should be created." using (tx.outputs.size == 1)
                    val out = tx.outputsOfType<TradeState>().single()
                    "The creating party and the counter party cannot be the same entity." using (out.initiatingParty != out.counterParty)
                    "All of the participants must be signers." using (command.signers.containsAll(out.participants.map { it.owningKey }))

                     // Trade-specific constraints.
                    "The sell currency and the buy currency cannot be the same entity." using (out.sellCurrency != out.buyCurrency)
                    "The Trade's sell value must be non-negative." using (out.sellValue > 0)
                    "The Trade's buy value must be non-negative." using (out.buyValue > 0)
                    "The Trade's sell currency can't be empty." using (out.sellCurrency.isNotEmpty())
                    "The Trade's buy currency can't be empty." using (out.buyCurrency.isNotEmpty())
                }
            }
```
The countertrade agreement is similar to the previous one.

### Client

The client class is just an example code of how to connect to the node using RPC and Kotlin. You can test it using Gradle by executing the task ``runExampleClientRPCKotlin``.

### Plugin

The plugin is used to register our API in our web application.

### Flow

The flow part defines the flow of the transactions on two sides: trader and countertrader. Both are very similar and has a ``call`` method that go through all 5 stages of the transactions:

1. Get the notary and generate the transaction based on the current state.
```java
val notary = serviceHub.networkMapCache.notaryIdentities[0]
progressTracker.currentStep = GENERATING_TRANSACTION
...
```

2. Verify the transaction.
```java
txBuilder.verify(serviceHub)
```

3. Sign the transaction.
```java
val partSignedTx = serviceHub.signInitialTransaction(txBuilder)
```

4. Send the transaction to the counterparty.
```java
val otherPartyFlow = initiateFlow(counterParty)
val fullySignedTx = subFlow(CollectSignaturesFlow(partSignedTx, setOf(otherPartyFlow), GATHERING_SIGS.childProgressTracker()))
```

5. Save the transaction after it's signed by the counterparty.
```java
return subFlow(FinalityFlow(fullySignedTx, FINALISING_TRANSACTION.childProgressTracker()))
```

It's easy to understand it when we see it in our web application.

### API

The API part is in fact our web application implementation. An example of the ``PUT`` request on ``create-trade`` path. It use the RCP to connect to the node with ``startTrackedFlow`` method. In the API some basic validation is done in the first place. The code can be found in 
``cordapp-trading\kotlin-source\src\main\kotlin\com\trading\api\TradeApi.kt``.

```java
@PUT
@Path("create-trade")
fun createTrade(@QueryParam("sellValue") sellValue: Int,@QueryParam("sellCurrency") sellCurrency: String,@QueryParam("buyValue") buyValue: Int,@QueryParam("buyCurrency") buyCurrency: String, @QueryParam("counterParty") counterParty: CordaX500Name?,@QueryParam("tradeStatus") tradeStatus: String): Response {
       
       if (counterParty == null) {
            return Response.status(BAD_REQUEST).entity("Query parameter 'Counter partyName' missing or has wrong format.\n").build()
        }
        if (sellValue <= 0 ) {
            return Response.status(BAD_REQUEST).entity("Query parameter 'Sell Value' must be non-negative.\n").build()
        }
        if (buyValue <= 0 ) {
            return Response.status(BAD_REQUEST).entity("Query parameter 'Buy Value' must be non-negative.\n").build()
        }
        val counterParty = rpcOps.wellKnownPartyFromX500Name(counterParty) ?:
                return Response.status(BAD_REQUEST).entity("Counter Party named $counterParty cannot be found.\n").build()

        return try {
            val signedTx = rpcOps.startTrackedFlow(::Initiator, sellValue,sellCurrency,buyValue,buyCurrency,tradeStatus, counterParty).returnValue.getOrThrow()
            Response.status(CREATED).entity("Transaction id ${signedTx.id} committed to ledger.\n").build()

        } catch (ex: Throwable) {
            logger.error(ex.message, ex)
            Response.status(BAD_REQUEST).entity(ex.message!!).build()
        }
    }
```

It's important to mention that the web application static files are placed in the resource directory: ``cordapp-trading/kotlin-source/src/main/resources/tradingWeb/``.

## Execute

To run the nodes we do it similar as in the previous notebook, but this time the ``runnoded`` script is placed in different location.

In [2]:
!cd /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/ && ./runnodes

Starting nodes in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes
Starting corda.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/PartyC on debug port 5005
Starting corda-webserver.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/PartyC on debug port 5006
Starting corda.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/Notary on debug port 5007
Starting corda.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/PartyB on debug port 5008
Starting corda-webserver.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/PartyB on debug port 5009
Starting corda.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/PartyA on debug port 5010
Starting corda-webserver.jar in /home/codete/workshop/cordapp-trading/kotlin-source/build/nodes/PartyA on debug port 5011
Started 7 processes
Finished starting nodes


## Web application

It can take about 60 seconds to run all 7 processes. Three process are web applications on each node, excluding the notary. You can access the applications by using the previously defined web ports on localhost like: [http://localhost:10021/](http://localhost:10021/). The other ports are 10022 and 10023. You can see that there are some paths that are defined in the API. You can go to [http://localhost:10021/web/trading/](http://localhost:10021/web/trading/) and trade.